In [1]:
import numpy as np
#import h5py
import time
import copy
from random import randint
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import autograd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

transform_test = transforms.Compose([
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
batch_size = 128
testset = torchvision.datasets.CIFAR10(root='./', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)
testloader = enumerate(testloader)

cuda:0


In [6]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()               # input dim
        self.conv1 = self.conv(3  ,196, 3,1,1, (196,32,32)) # 32*32
        self.conv2 = self.conv(196,196, 3,2,1, (196,16,16)) # 32*32
        self.conv3 = self.conv(196,196, 3,1,1, (196,16,16)) # 16*16
        self.conv4 = self.conv(196,196, 3,2,1, (196,8,8))   # 16*16
        self.conv5 = self.conv(196,196, 3,1,1, (196,8,8))   # 8*8
        self.conv6 = self.conv(196,196, 3,1,1, (196,8,8))   # 8*8
        self.conv7 = self.conv(196,196, 3,1,1, (196,8,8))   # 8*8
        self.conv8 = self.conv(196,196, 3,2,1, (196,4,4))   # 8*8
        self.pool = nn.MaxPool2d(4,4)           # 4*4
        
        self.fc1 = nn.Linear(196, 1)            # 1     critic output
        self.fc10 = nn.Linear(196, 10)          # 10    auxiliary classifier output
        
    def conv(self,i,o,k,s,p,ln_dim):
        return nn.Sequential(
            nn.Conv2d(i,o,k,s,p),
            nn.LayerNorm(ln_dim),
            nn.LeakyReLU()
        )
        
    def forward(self, x, extract_features=3):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        if(extract_features==3):
            h = F.max_pool2d(x,16,16)
            h = h.view(-1, 196)
            return h
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        if(extract_features==6):
            h = F.max_pool2d(x,8,8)
            h = h.view(-1, 196)
            return h
        x = self.conv7(x)
        x = self.conv8(x)
        if(extract_features==8):
            h = F.max_pool2d(x,4,4)
            h = h.view(-1, 196)
            return h
        x = self.pool(x)
        x1 = x.view(-1, 196)
        critic = self.fc1(x1)
        x2 = x.view(-1, 196)
        classifier = self.fc10(x2)
        return critic,classifier




In [7]:

model = torch.load('cifar10.model')
model.to(device)
model.eval()

C:\Users\wangwang\Anaconda3\lib\site-packages\torch\serialization.py:391: UserWarning: Couldn't retrieve source code for container of type discriminator. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


discriminator(
  (conv1): Conv2d(3, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (ln1): LayerNorm(torch.Size([196, 32, 32]), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm(torch.Size([196, 16, 16]), eps=1e-05, elementwise_affine=True)
  (ln3): LayerNorm(torch.Size([196, 16, 16]), eps=1e-05, elementwise_affine=True)
  (ln4): LayerNorm(torch.Size([196, 8, 8]), eps=1e-05, elementwise_affine=True)
  (ln5): LayerNorm(torch.Size

In [8]:
batch_idx, (X_batch, Y_batch) = testloader.__next__()
X_batch = Variable(X_batch,requires_grad=True).to(device)
Y_batch = Variable(Y_batch).to(device)

X = X_batch.mean(dim=0)
X = X.repeat(10,1,1,1)

Y = torch.arange(10).type(torch.int64)
Y = Variable(Y).to(device)

In [9]:
def plot(samples):
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(10, 10)
    gs.update(wspace=0.02, hspace=0.02)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample)
    return fig

X = X_batch.mean(dim=0)
X = X.repeat(batch_size,1,1,1)

Y = torch.arange(batch_size).type(torch.int64)
Y = Variable(Y).cuda()

lr = 0.1
weight_decay = 0.001
for i in range(200):
    output = model(X)

    loss = -output[torch.arange(batch_size).type(torch.int64),torch.arange(batch_size).type(torch.int64)]
    gradients = torch.autograd.grad(outputs=loss, inputs=X,
                              grad_outputs=torch.ones(loss.size()).cuda(),
                              create_graph=True, retain_graph=False, only_inputs=True)[0]

    prediction = output.data.max(1)[1] # first column has actual prob.
    accuracy = ( float( prediction.eq(Y.data).sum() ) /float(batch_size))*100.0
    print(i,accuracy,-loss)

    X = X - lr*gradients.data - weight_decay*X.data*torch.abs(X.data)
    X[X>1.0] = 1.0
    X[X<-1.0] = -1.0

## save new images
samples = X.data.cpu().numpy()
samples += 1.0
samples /= 2.0
samples = samples.transpose(0,2,3,1)

fig = plot(samples[0:100])
plt.savefig('visualization/max_features_3_nogen.png', bbox_inches='tight')
plt.close(fig)

0 0.78125 tensor([ 0.1600,  0.5515, -0.0304,  0.0971,  0.1485,  0.8352,  0.2309,  0.4800,
         1.0260,  0.2841,  0.2935,  0.3884,  0.0744,  0.2663,  1.2814,  0.1313,
         0.0747,  0.4275,  0.0240,  1.0478,  0.5160,  0.4275,  0.1821,  0.2287,
         0.5807,  0.6974,  0.0136,  0.9465,  0.4397,  0.8264, -0.0286,  0.3109,
         0.3100,  0.2869,  0.1428,  0.6627,  0.0165,  0.3666,  0.1311,  0.3438,
         0.3923,  0.6663,  0.0116, -0.1954, -0.1212,  0.5544,  0.7517,  0.1435,
         0.2800,  0.3628,  0.2768,  0.8097,  0.2459,  0.0892,  0.6718,  0.5567,
         0.3070,  1.3074,  0.4984,  0.4151,  0.2285,  0.6249, -0.3365,  0.6400,
         0.5185,  0.3622,  0.6491,  0.0143,  0.1285,  0.1366,  0.3602,  0.2588,
         0.9772,  0.3313,  0.4285,  0.0919,  0.4601,  0.3907,  0.1508,  0.8387,
         0.2585,  0.9499,  0.1844,  0.1764,  0.7574,  0.2407,  0.1819,  0.2135,
         0.3352,  0.7822,  0.1815,  0.3906,  0.8287,  0.4920,  0.5841,  0.4733,
         0.2530,  1.7475,  0.0

10 20.3125 tensor([0.7146, 1.7826, 1.0494, 0.2061, 0.3669, 1.2862, 4.1110, 1.3053, 1.8430,
        3.5525, 0.7972, 1.7217, 0.5639, 2.7016, 1.7500, 0.6417, 0.3320, 1.3974,
        1.1786, 1.7429, 0.9515, 0.9881, 3.1033, 0.6966, 2.2901, 1.4518, 0.5573,
        3.6367, 0.9706, 1.7974, 1.3091, 1.4697, 1.7866, 3.0476, 0.5310, 1.0958,
        0.3980, 0.6911, 1.3341, 1.1504, 1.7360, 0.9132, 0.6754, 0.1641, 0.0643,
        1.1287, 1.7988, 1.0808, 0.6945, 0.6765, 1.7053, 3.4635, 0.4565, 0.6807,
        2.2043, 1.8998, 1.1320, 2.0814, 1.3298, 2.0209, 9.4969, 1.6509, 0.1264,
        6.7585, 1.3505, 1.7335, 1.0617, 0.4313, 0.5849, 0.6550, 0.7091, 0.7557,
        1.8428, 1.7781, 0.9329, 0.4348, 3.5709, 1.4435, 4.0237, 2.2566, 0.9560,
        1.9496, 0.4719, 1.1208, 2.3313, 1.2093, 0.7483, 2.0092, 0.7588, 1.8890,
        0.9393, 3.2945, 1.6791, 1.2368, 0.9715, 1.5620, 1.3868, 2.7948, 0.6493,
        1.5573, 2.6658, 0.4744, 1.0706, 0.9768, 1.1914, 1.0765, 1.2522, 1.5705,
        2.0168, 0.7876, 1.563

20 37.5 tensor([ 1.2673,  3.0090,  2.1246,  0.3147,  0.5841,  1.7358,  7.8090,  2.1270,
         2.6569,  6.8041,  1.2993,  3.0491,  1.0517,  5.1251,  2.2167,  1.1505,
         0.5881,  2.3643,  2.3276,  2.4359,  1.3854,  1.5473,  6.0097,  1.1635,
         3.9601,  2.2038,  1.0991,  6.1513,  1.4996,  2.7652,  2.6423,  2.6227,
         3.2560,  5.7954,  0.9182,  1.5277,  0.7787,  1.0149,  2.4950,  1.9535,
         3.0741,  1.1590,  1.3364,  0.5227,  0.2490,  1.7014,  2.8419,  2.0146,
         1.1067,  0.9887,  3.1289,  6.1008,  0.6667,  1.2706,  3.7288,  3.2386,
         1.9544,  2.8523,  2.1580,  3.6187, 16.7954,  2.6722,  0.5883, 12.5118,
         2.1794,  3.0999,  1.4734,  0.8467,  1.0405,  1.1719,  1.0571,  1.2510,
         2.7045,  3.2189,  1.4358,  0.7763,  6.6646,  2.4927,  7.4209,  3.6678,
         1.6507,  2.9465,  0.7588,  2.0611,  3.8971,  2.1749,  1.3133,  3.7960,
         1.1814,  2.9921,  1.6925,  6.1833,  2.5275,  1.9794,  1.3574,  2.6466,
         2.5153,  3.8385,  1.254

30 39.0625 tensor([ 1.8171,  4.2000,  3.1752,  0.4228,  0.7998,  2.1833, 10.5474,  2.9429,
         3.4669,  9.5766,  1.7988,  4.1955,  1.5367,  7.3166,  2.6814,  1.6564,
         0.8426,  3.3261,  3.4341,  3.1254,  1.8174,  2.1036,  8.6348,  1.6284,
         5.4465,  2.9516,  1.6374,  7.6534,  2.0256,  3.7279,  3.9666,  3.6261,
         4.6635,  8.2788,  1.3035,  1.9579,  1.1580,  1.3376,  3.4521,  2.7504,
         4.3700,  1.4037,  1.9890,  0.8798,  0.4328,  2.2712,  3.8793,  2.8710,
         1.5110,  1.2994,  4.5395,  8.1147,  0.8761,  1.8574,  5.0196,  4.5687,
         2.7643,  3.6190,  2.9809,  4.9628, 21.4260,  3.6862,  1.0482, 16.7069,
         3.0031,  4.4573,  1.8832,  1.2599,  1.4943,  1.6857,  1.4036,  1.7439,
         3.5599,  4.6495,  1.9362,  1.1160,  9.1866,  3.5351,  9.4611,  5.0219,
         2.3411,  3.9383,  1.0446,  2.9774,  5.3088,  3.1357,  1.8752,  5.2566,
         1.6023,  4.0882,  2.3677,  8.6912,  3.3716,  2.7178,  1.7413,  3.7248,
         3.5737,  4.8760,  1.

40 40.625 tensor([ 2.3626,  5.1585,  4.0551,  0.5304,  1.0135,  2.6282, 12.3282,  3.7036,
         4.2715, 11.3757,  2.2946,  5.2138,  2.0091,  9.0685,  3.1433,  2.1187,
         1.0835,  4.2805,  4.3271,  3.7826,  2.2471,  2.6559, 10.3644,  2.0902,
         6.5051,  3.6803,  2.0897,  8.6313,  2.5156,  4.6500,  5.2778,  4.2626,
         5.5954, 10.3193,  1.6857,  2.3855,  1.5351,  1.6586,  4.3433,  3.3605,
         5.3604,  1.6466,  2.5550,  1.2348,  0.6154,  2.8160,  4.9084,  3.6142,
         1.8406,  1.6080,  5.8046,  9.0446,  1.0846,  2.4397,  5.9231,  5.8314,
         3.4877,  4.3797,  3.7591,  5.8856, 24.4114,  4.5625,  1.5050, 19.4676,
         3.7178,  5.7236,  2.2901,  1.6699,  1.9454,  2.1477,  1.7481,  2.2334,
         4.2094,  5.9435,  2.4331,  1.4533, 11.0813,  4.4802, 10.3198,  6.1368,
         3.0244,  4.9224,  1.3288,  3.8264,  6.4696,  4.0310,  2.4324,  6.2804,
         2.0209,  5.0791,  2.8876, 10.2534,  4.2093,  3.4502,  2.1225,  4.7066,
         4.4505,  5.8632,  2.4

...

150 35.15625 tensor([ 4.8498,  9.2973,  7.0101,  1.4019,  2.0665,  5.8115, 16.4970,  6.1172,
         9.1431, 16.0299,  4.6802,  8.4148,  4.0952, 14.5786,  6.3780,  4.3501,
         2.3896,  9.0732,  7.0570,  7.8473,  5.2213,  5.6107, 15.6490,  4.7758,
        10.2347,  6.9953,  4.4639, 11.8327,  5.0464,  9.4369, 11.9706,  6.2228,
         8.6306, 16.4890,  3.5501,  5.1267,  3.9757,  4.2033,  7.4277,  5.2803,
         8.8067,  3.2383,  4.2849,  3.4608,  1.9343,  5.8932,  9.8841,  6.5045,
         3.7717,  3.6469, 11.0685, 11.6867,  2.7709,  5.2266,  9.0893, 10.5950,
         6.2871,  8.4359,  6.7536,  8.9469, 30.1104,  7.4408,  4.0729, 25.6470,
         7.2409, 10.2534,  4.5321,  3.4462,  4.9263,  4.5029,  4.2719,  5.1386,
         7.7802, 10.4749,  5.4125,  3.4665, 15.8092,  8.1896, 12.5364,  9.9937,
         6.0905,  9.6137,  3.2785,  6.7092, 10.5646,  9.2454,  4.6522,  9.5537,
         5.0679,  9.8480,  5.2132, 12.6340,  8.2281,  6.8565,  4.2456,  9.5372,
         8.1555, 10.4588,  

160 34.375 tensor([ 4.9288,  9.4012,  7.0840,  1.4442,  2.1034,  5.9506, 16.6000,  6.1751,
         9.3119, 16.1049,  4.7720,  8.4995,  4.1483, 14.7069,  6.5082,  4.4286,
         2.4454,  9.2415,  7.1473,  8.0242,  5.3309,  5.7024, 15.8259,  4.8675,
        10.3376,  7.1001,  4.5496, 11.9145,  5.1299,  9.5887, 12.4988,  6.2863,
         8.7016, 16.6146,  3.6069,  5.2107,  4.0533,  4.3114,  7.5185,  5.3176,
         8.9189,  3.3161,  4.3244,  3.5303,  1.9915,  6.0013, 10.0536,  6.5851,
         3.8566,  3.7181, 11.2322, 11.7435,  2.8494,  5.2987,  9.1433, 10.7144,
         6.3497,  8.5797,  6.8309,  9.0345, 30.2447,  7.4932,  4.1619, 25.7737,
         7.3556, 10.3906,  4.6129,  3.5055,  5.0457,  4.5862,  4.3768,  5.2597,
         7.9118, 10.5947,  5.5035,  3.5455, 15.8738,  8.2619, 12.5852, 10.1158,
         6.1943,  9.7407,  3.3594,  6.7829, 10.6741,  9.3994,  4.7021,  9.6203,
         5.1853, 10.0131,  5.2863, 12.6966,  8.3575,  6.9605,  4.3250,  9.6923,
         8.2421, 10.6220,  6.

170 33.59375 tensor([ 4.9932,  9.5008,  7.1540,  1.4860,  2.1362,  6.0737, 16.6898,  6.2322,
         9.4686, 16.1706,  4.8600,  8.5681,  4.1969, 14.8249,  6.6233,  4.4954,
         2.4938,  9.3897,  7.2253,  8.1845,  5.4310,  5.7739, 15.9854,  4.9397,
        10.4345,  7.1918,  4.6179, 11.9838,  5.2099,  9.7257, 12.9813,  6.3414,
         8.7647, 16.7208,  3.6590,  5.2825,  4.1221,  4.4056,  7.5971,  5.3536,
         9.0162,  3.3865,  4.3623,  3.5939,  2.0448,  6.0962, 10.2029,  6.6499,
         3.9251,  3.7841, 11.3738, 11.7982,  2.9207,  5.3634,  9.1921, 10.8203,
         6.4091,  8.7184,  6.9061,  9.1079, 30.3461,  7.5410,  4.2327, 25.8933,
         7.4613, 10.5242,  4.6928,  3.5618,  5.1506,  4.6541,  4.4781,  5.3673,
         8.0194, 10.6983,  5.5870,  3.6050, 15.9356,  8.3300, 12.6296, 10.2300,
         6.2848,  9.8554,  3.4320,  6.8493, 10.7721,  9.5299,  4.7478,  9.6809,
         5.2948, 10.1614,  5.3534, 12.7525,  8.4727,  7.0513,  4.3864,  9.8368,
         8.3210, 10.7680,  

180 34.375 tensor([ 5.0495,  9.5932,  7.2206,  1.5241,  2.1643,  6.1831, 16.7774,  6.2870,
         9.6143, 16.2289,  4.9427,  8.6244,  4.2397, 14.9272,  6.7252,  4.5594,
         2.5328,  9.5328,  7.2980,  8.3159,  5.5133,  5.8278, 16.1256,  5.0016,
        10.5236,  7.2676,  4.6829, 12.0431,  5.2850,  9.8486, 13.3642,  6.3930,
         8.8201, 16.8085,  3.7071,  5.3439,  4.1830,  4.4883,  7.6654,  5.3784,
         9.0952,  3.4511,  4.3998,  3.6526,  2.0894,  6.1844, 10.3446,  6.6978,
         3.9833,  3.8402, 11.4993, 11.8481,  2.9875,  5.4272,  9.2336, 10.9199,
         6.4651,  8.8477,  6.9708,  9.1658, 30.4319,  7.5836,  4.2920, 25.9964,
         7.5503, 10.6503,  4.7713,  3.6161,  5.2413,  4.7153,  4.5702,  5.4541,
         8.1129, 10.7890,  5.6650,  3.6486, 15.9931,  8.3929, 12.6688, 10.3346,
         6.3703,  9.9545,  3.4984,  6.9048, 10.8554,  9.6505,  4.7879,  9.7339,
         5.3961, 10.2995,  5.4100, 12.8019,  8.5634,  7.1384,  4.4394,  9.9603,
         8.3889, 10.8959,  6.

190 33.59375 tensor([ 5.1035,  9.6710,  7.2790,  1.5501,  2.1918,  6.2692, 16.8594,  6.3337,
         9.7397, 16.2832,  5.0170,  8.6798,  4.2798, 15.0205,  6.8251,  4.6164,
         2.5691,  9.6684,  7.3550,  8.4309,  5.5868,  5.8736, 16.2430,  5.0518,
        10.6019,  7.3314,  4.7397, 12.0944,  5.3507,  9.9601, 13.6825,  6.4324,
         8.8659, 16.8902,  3.7513,  5.4029,  4.2386,  4.5610,  7.7303,  5.4003,
         9.1665,  3.5054,  4.4363,  3.7076,  2.1256,  6.2674, 10.4745,  6.7394,
         4.0373,  3.8931, 11.6125, 11.8896,  3.0430,  5.4813,  9.2693, 11.0142,
         6.5153,  8.9733,  7.0304,  9.2171, 30.5056,  7.6231,  4.3396, 26.0858,
         7.6279, 10.7673,  4.8447,  3.6650,  5.3217,  4.7713,  4.6505,  5.5323,
         8.1964, 10.8702,  5.7369,  3.6848, 16.0428,  8.4456, 12.7033, 10.4338,
         6.4533, 10.0395,  3.5561,  6.9468, 10.9340,  9.7603,  4.8265,  9.7796,
         5.4851, 10.4259,  5.4611, 12.8455,  8.6455,  7.2174,  4.4839, 10.0607,
         8.4511, 11.0069,  